**Authors: Shreyas,Shrushti**

Model Trained on Nvidia A-100 80 Gb Gpu.

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
#breakdown.py
import os
import threading
import pandas as pd
import numpy as np
import time


'''
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))
'''
sponsor_times = pd.read_csv('/content/drive/MyDrive/PROJECT/0part.csv', sep=',')   #--> spons sub dataset
sponsor_times.head()

number_of_entries = len(sponsor_times['category'])

for category in set(sponsor_times['category']):
    count = np.count_nonzero(sponsor_times['category'] == category)
    print(f'{category}: {count}/{number_of_entries} ({count / number_of_entries * 100})')

<ipython-input-4-a8bcd83dd6ba>:14: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  sponsor_times = pd.read_csv('/content/drive/MyDrive/PROJECT/0part.csv', sep=',')   #--> spons sub dataset


filler: 33396/3699847 (0.9026319196442447)
exclusive_access: 1134/3699847 (0.030649916064096706)
sponsor: 1194642/3699847 (32.28895681361959)
outro: 613052/3699847 (16.569658150728937)
preview: 65203/3699847 (1.7623161173962059)
poi_highlight: 90940/3699847 (2.4579394769567497)
moreCategories: 4/3699847 (0.00010811257870933582)
selfpromo: 322032/3699847 (8.703927486731208)
intro: 886386/3699847 (23.957369047963333)
music_offtopic: 152804/3699847 (4.130008619275338)
interaction: 340254/3699847 (9.196434339041588)


In [ ]:
#keep vid id
import pandas as pd

# Filters the csv so that only the videoID column is preserved.
sponsor_times = pd.read_csv('/content/drive/MyDrive/PROJECT/0part.csv', sep=',')  #--> spons sub dataset (same as before)
video_ids = sponsor_times[['videoID']]
video_ids = video_ids[sponsor_times['category'] == 'sponsor']
video_ids.to_csv('/content/0part_new.csv') #temp csv will be saved in your LOCAL Runtime

<ipython-input-5-61072a453b3b>:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  sponsor_times = pd.read_csv('/content/drive/MyDrive/PROJECT/0part.csv', sep=',')  #--> spons sub dataset (same as before)


In [ ]:
from typing import List
class DBSegment:
	category: str
	startTime: float
	endTime: float
	UUID: str
	userID: str
	votes: int
	views: int
	locked: int
	hidden: int
	shadowHidden: int
	videoID: str
	videoDuration: int
	reputation: int
	hashedVideoID: str
	timeSubmitted: int
	userAgent: str
	service: str
	description: str

class OverlappingSegmentGroup:
	def __init__(self):
		self.segments: List[DBSegment] = []
		self.votes = 0

def build_segment_groups(segments: List[DBSegment]) -> List[OverlappingSegmentGroup]:
	"""
	This function will find segments that are contained inside of eachother, called similar segments.  (Timestamps)
	Segments with less than -1 votes are already ignored before this function is called.

	Based on https://github.com/ajayyy/SponsorBlockServer/blob/e74b985304443b17b429c5c82696c7a03e78a166/src/routes/getSkipSegments.ts#L276
	"""

	# Create groups of segments that are similar to eachother
	# Segments must be sorted by their startTime so that we can build groups chronologically:
	# 1. As long as the segments' startTime fall inside the currentGroup, we keep adding them to that group
	# 2. If a segment starts after the end of the currentGroup (> cursor), no other segment will ever fall
	#    inside that group (because they're sorted) so we can create a new one
	overlappingSegmentsGroups: List[OverlappingSegmentGroup] = []
	currentGroup = None
	cursor = -1 # -1 to make sure that, even if the 1st segment starts at 0, a new group is created
	for segment in segments:
		if segment.startTime >= cursor:
			currentGroup = OverlappingSegmentGroup()
			overlappingSegmentsGroups.append(currentGroup)

		currentGroup.segments.append(segment)
		# only if it is a positive vote, otherwise it is probably just a sponsor time with slightly wrong time
		if segment.votes > 0:
			currentGroup.votes += segment.votes

		cursor = max(cursor, segment.endTime)

	return overlappingSegmentsGroups

def get_best_segment(group: OverlappingSegmentGroup):
	"""
	SponsorBlock chooses a segment from an overlap group randomly by using the
	votes property as a weight. This is done so that all segments can have a
	chance of appearing and makes sense in that system, but here we just
	want the best possible match, hence we pick the segment with the highest vote.
	"""

	return max(group.segments, key=lambda segment: segment.votes)

In [ ]:
#main.py TRYYYYYYYYYYYYYYYYYYYYYYy------------------222222222222222222
from concurrent.futures import thread
from glob import glob
import os
import sys
import multiprocessing.dummy
import subprocess
from sys import stdout
import threading
import pandas as pd
import time
import numpy as np
import datetime

import requests, json
import matplotlib.pyplot as plt



# File with column 'videoID'
INPUT_FILENAME = '/content/0part_new.csv'                                           #'/content/testpartiii.csv'
# File to use to save job state
STATUS_FILENAME = '/content/drive/MyDrive/CNN_Dataset_Dump/shreyas/slot_1/statuscnn.csv' #------> change location as per your drive and slot folder!
                                           # status csv to save job state. if loop fails and u hv to restart then you will be able to restart frm where u left!!
# Number of concurrent yt-dlp scripts to run
NUM_THREADS = int('32')

iteration=0
chunk_id=0
processed_df=None

# Runtime stats to display progress
processed = 0
gold = 0
silver = 0
start = time.time()
# Synchronisation for the variables above and the STATUS_FILENAME
lock = threading.Lock()
sponsor_times = '/content/drive/MyDrive/CNN_Dataset_Dump/shreyas/0part.csv'  #------> spons sub - dataset (same as first cell)
sponsorml_df = pd.read_csv(sponsor_times)                                          #FF
grouped_df = sponsorml_df.groupby(by=["videoID"])

try:
    # Try to restore the state from last time
    results = pd.read_csv(STATUS_FILENAME, index_col=False)
    print('Continuing...')
except:
    results = pd.DataFrame(columns=['videoID', 'status'])      #'Intensity_Score''TimeRangeStart''Marker_Duration'

processed_last_session = len(results['videoID'])


def vid_process(vid_id):

        segments = grouped_df.get_group(vid_id)   #get sponsor segment time stamps fetched from the main public dataset.
        if len(segments) > 0:
          print('first if')
          segments = [segment for _, segment in segments.iterrows()
          if segment.category == 'sponsor' and segment.votes >= 1]
          # Filter out similar timestamps
          segments = [get_best_segment(group) for group in build_segment_groups(segments)]
          segment_times = [(round(segment.startTime,0), round(segment.endTime,0)) for segment in segments]
          print('got seg times',len(segment_times))
          # ----------------------------LOGIC-------------------------------------------------------------------------------------------------------
          if len(segment_times) > 0:
            final_sponsor_segments=[]
            for i in range(len(segment_times)):
                print('got secfor')
                temp=1
                for j in range(0,len(segment_times[i]),2):
                  start_spons = segment_times[i][j]
                  start_spons = int(round(start_spons,0))
                  print('start_spons is',start_spons)

                for k in range(1,len(segment_times[i]),2):
                  end_spons = segment_times[i][k]
                  end_spons = int(round(end_spons,0))
                  print('end_spons is',end_spons)

                s = end_spons-start_spons
                sby2 = int(round(s/2,0))
                start_prespons = max(start_spons-sby2,0)
                start_prespons = int(round(start_prespons,0))
                end_prespons = max(int(round(start_spons-1)),0)
                start_postspons = int(round(end_spons+1))
                end_postspons = end_spons+sby2
                end_postspons = int(round(end_postspons,0))

                sub1 = end_prespons - start_prespons
                sub2 = end_postspons - start_postspons
                vid_id1 = str(vid_id)
                if sub1 < sby2:
                 end_postspons = end_postspons + sby2 - sub1

                if  sub2 < sby2:
                  start_prespons = start_prespons - sby2 + sub2
                                                    #--------------------------------------------------------------------CHANGE FOLDER SAVE LOCATION FOR ALL 3 -------------------------------------------------------------------here-------------------------------
                str_command1 = f''' ! ffmpeg -ss {start_prespons} -to {end_prespons} -i $(yt-dlp -f worstvideo --get-url "https://www.youtube.com/watch?v={vid_id1}") -vf fps=1 /content/drive/MyDrive/CNN_Dataset_Dump/shreyas/slot_1/non_sponsor/{vid_id1}_Prenonspons%d.jpg > /dev/null 2>&1 '''
                os.system(str_command1)
                str_command2 = f''' ! ffmpeg -ss {start_spons} -to {end_spons} -i $(yt-dlp -f worstvideo --get-url "https://www.youtube.com/watch?v={vid_id1}") -vf fps=1 /content/drive/MyDrive/CNN_Dataset_Dump/shreyas/slot_1/sponsor/{vid_id1}_Spons%d.jpg > /dev/null 2>&1 '''
                os.system(str_command2)
                str_command3 = f''' ! ffmpeg -ss {start_postspons} -to {end_postspons} -i $(yt-dlp -f worstvideo --get-url "https://www.youtube.com/watch?v={vid_id1}") -vf fps=1 /content/drive/MyDrive/CNN_Dataset_Dump/shreyas/slot_1/non_sponsor/{vid_id1}_Postnonspons%d.jpg > /dev/null 2>&1 '''
                os.system(str_command3)
                                                    #-------------------------------------------------------------------CHANGE FOLDER SAVE LOCATION FOR ALL 3 --------------------------------------------------------------------here------------------------------
            yield 'gold'

          else:
              yield 'silver'

        else:
           yield 'silver'


start1 = time.time()
def process_video(vid_id):
    """
    Downloads the captions for the video to the output
    path save the job state to disk.
    """
    global gold
    global silver
    global processed

    try:
        status = vid_process(vid_id)
        status = next(status)
        print(status)
        if status == 'gold':
            gold += 1
        if status == 'silver':
            silver += 1
        with lock:
            results.loc[len(results.index)] = [vid_id, status]
    except Exception as e:
        print(e)
        return
    processed += 1
    elapsed = time.time() - start1
    print()
    print(f"""{processed} videos processed at a rate of {(elapsed / processed):.2f}s per video
        {(gold / processed * 100):.2f}% of videos have Frames, {(silver / processed * 100):.2f}% of videos don't have Frames
        {((processed + processed_last_session) / len(video_ids) * 100):.2f}% done""")

sponsor_times = pd.read_csv(INPUT_FILENAME, sep=',', index_col=False)
video_ids = sponsor_times['videoID'].unique()
np.random.shuffle(video_ids)
video_ids = list(video_ids)

print()
print(f'Found {len(video_ids)} videos')

if processed_last_session > 0:
    print(f' -->{processed_last_session} videos already processed.');
    video_ids = set(video_ids).difference(set(results['videoID']))                 #2nd try error

print(f' -->{len(video_ids)} videos remaining')

#p = multiprocessing.dummy.Pool(processes=32)

start = time.time()
def _print_progress():
    while True:
        if processed == 0:
            print('.', end='')
            time.sleep(1)
            continue
        #--cut
        with lock:
            results.to_csv(STATUS_FILENAME + '~', index=False)
            os.replace(STATUS_FILENAME + '~', STATUS_FILENAME)
        time.sleep(5)

timer = threading.Thread(target=_print_progress)
timer.start()



video_ids = list(video_ids)
for i in range(20000): #no of video ID's to iterate in one sitting.
  process_video(video_ids[i])

Continuing...

Found 754708 videos
 -->2 videos already processed.
 -->754707 videos remaining
.first if
got seg times 1
got secfor
start_spons is 0
end_spons is 61
.................................................gold

1 videos processed at a rate of 10.12s per video
        100.00% of videos have Intrest Graph, 0.00% of videos don't have Intrest Graph
        0.00% done


In [ ]:
#Processed around 60k videos. Frames were captured at 1fps. For a vid, amt of frames captured were equal for both classes.
#Eg: len sponsor seg = 20 secs then amt frames captured for both sponsor and non_sponsor is 20 each. Total = 20+20 frames: sponsor+non sponsor[pre+post]
#___---pre-non-spons(10secs)--- ~spons (20secs)~ ---post-non-spons(10secs)---___________

In [ ]:
#!du -h FINAL_CNN          #verifying dataset size

30G	FINAL_CNN/Train/sponsor
29G	FINAL_CNN/Train/non_sponsor
58G	FINAL_CNN/Train
3.7G	FINAL_CNN/Valid/sponsor
3.7G	FINAL_CNN/Valid/non_sponsor
7.3G	FINAL_CNN/Valid
3.9G	FINAL_CNN/Test/sponsor
3.9G	FINAL_CNN/Test/non_sponsor
7.7G	FINAL_CNN/Test
73G	FINAL_CNN


In [ ]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
from PIL import Image, ImageFile

In [ ]:
## Now, we import timm, torchvision image models
#!pip install timm
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss

In [ ]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
from tqdm import tqdm
import time
import copy

In [ ]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [ ]:
def check_Image(path):
  try:
    im = Image.open(path)
    return True
  except:
    return False

In [ ]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.3, value='random') #0.2
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "Train/"), transform = transform, is_valid_file=check_Image)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=5)
        print('trainload')
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "Valid/"), transform=transform, is_valid_file=check_Image)
        print('vald')
        test_data = datasets.ImageFolder(os.path.join(data_dir, "Test/"), transform=transform, is_valid_file=check_Image)
        print('testd')
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=5)
        print('valload')
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
        print('testload')
        return val_loader, test_loader, len(val_data), len(test_data)

In [ ]:
current_folder = globals()['_dh'][0]

# Calculating path to the input data
data_location = os.path.join(current_folder,'FINAL_CNN/')

In [ ]:
current_folder = globals()['_dh'][0]

# Calculating path to the input data
data_location2 = os.path.join(current_folder,'FINAL_CNN/Train/')

In [ ]:
dataset_path = data_location

In [ ]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True) #495 512
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 64, train=False) #40
print('1')

trainload
vald
testd
valload
testload
1


In [ ]:
classes = get_classes(data_location2)
print(classes,'=', len(classes))

['non_sponsor', 'sponsor'] = 2


In [ ]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [ ]:
print(len(train_loader), len(val_loader), len(test_loader))

44856 11543 12011


In [ ]:
print(train_data_len, valid_data_len, test_data_len)

5741450 738752 768704


In [ ]:
# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model_parent = torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=True)

Using cache found in /home/fsuser/.cache/torch/hub/facebookresearch_deit_main


In [ ]:
for param in model_parent.parameters(): #IMPPPPPPPPPPPP
    param.requires_grad = False

n_inputs = model_parent.head.in_features
model_parent.head = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(2048, 1524),
    nn.ReLU(),
    nn.Linear(1524, 1024),
    nn.Dropout(0.16),
    nn.Linear(1024, 800),
    nn.Linear(800, 512),
    nn.Linear(512, 2)  #2 classes ----/
)
model2= model_parent.to(device)
print(model2.head)

Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.16, inplace=False)
  (7): Linear(in_features=1024, out_features=800, bias=True)
  (8): Linear(in_features=800, out_features=512, bias=True)
  (9): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
# Define the dropout layer to be added
dropout_layer = nn.Dropout(0.3)

# Get the reference to the existing layer
old_conv = model2.head[7]

# Create a new sequential module with the dropout layer followed by the existing layer
new_conv = nn.Sequential(
    old_conv, dropout_layer
)

# Update the model with the new sequential module
model2.head[7] = new_conv

print(model2.head)

Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.16, inplace=False)
  (7): Sequential(
    (0): Linear(in_features=1024, out_features=800, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
  (8): Linear(in_features=800, out_features=512, bias=True)
  (9): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
# Define the dropout layer to be added
new_layer = nn.Identity()

# Get the reference to the existing layer
old_conv = model2.head[1]

# Create a new sequential module with the dropout layer followed by the existing layer
new_conv = nn.Sequential(new_layer)

# Update the model with the new sequential module
model2.head[1] = new_conv

print(model2.head)

Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): Sequential(
    (0): Identity()
  )
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.16, inplace=False)
  (7): Sequential(
    (0): Linear(in_features=1024, out_features=800, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
  (8): Linear(in_features=800, out_features=512, bias=True)
  (9): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
for param in model_parent.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model_parent.head.in_features
model_parent.head = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.ReLU(),
    nn.Dropout(0.25),
    nn.Linear(2048, 1524),
    nn.ReLU(),
    nn.Linear(1524, 1024),
    nn.Dropout(0.15),
    nn.Linear(1024, 800),
    nn.Dropout(0.2),
    nn.Linear(800, 512),
    nn.Linear(512, 2)  #2 classes ----/
)
model2= model_parent.to(device)
print(model2.head)

Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.25, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.15, inplace=False)
  (7): Linear(in_features=1024, out_features=800, bias=True)
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=800, out_features=512, bias=True)
  (10): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
print(model2.head)

Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): Sequential(
    (0): Identity()
  )
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.16, inplace=False)
  (7): Sequential(
    (0): Linear(in_features=1024, out_features=800, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
  (8): Linear(in_features=800, out_features=512, bias=True)
  (9): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
# load the saved information back
checkpoint = torch.load('DeiT_dense_256batch_epoch_3.pt')
optimizer = torch.optim.AdamW(model2.head.parameters(), lr=0.001, weight_decay=0.1)
model2.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])



In [ ]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer.param_groups[0]['weight_decay'] = 0.1
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.1
)

In [ ]:
exp_lr_scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=1, threshold=0.0001, threshold_mode='abs')

In [ ]:
ite = 1
def train_model(model2, criterion, optimizer, scheduler, num_epochs=3):  #5epochs;
    global ite

    since = time.time()
    best_model_wts = copy.deepcopy(model2.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        model_save_name = 'DeiT_dense_128batch_epoch_' + str(ite) + '.pt'

        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model2.train() # model to training mode
            else:
                model2.eval() # model to evaluate

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model2(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step(loss) # step at end of epoch
                torch.save({'model_state_dict': model2.state_dict(),'optimizer_state_dict': optimizer.state_dict()
                           }, model_save_name)
                ite = ite+1

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))


            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model2.state_dict()) # keep the best validation accuracy model

        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    model2.load_state_dict(best_model_wts)
    return model2

In [ ]:
model_ft5 = train_model(model2, criterion, optimizer, exp_lr_scheduler2) #Epoch 1-7 and save

Epoch 0/6
----------


100%|███████████████████████████████████████████████████████████████████████████| 11214/11214 [1:22:12<00:00,  2.27it/s]


train Loss: 0.5325 Acc: 0.7582


100%|█████████████████████████████████████████████████████████████████████████████| 23086/23086 [13:08<00:00, 29.27it/s]


val Loss: 0.6118 Acc: 0.6787

Epoch 1/6
----------


...

100%|█████████████████████████████████████████████████████████████████████████████| 23086/23086 [13:11<00:00, 29.16it/s]


val Loss: 0.6098 Acc: 0.6864

Epoch 4/6
----------


 86%|█████████████████████████████████████████████████████████████████           | 9602/11214 [1:09:12<11:22,  2.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████| 11214/11214 [1:20:53<00:00,  2.31it/s]


train Loss: 0.4949 Acc: 0.7897


 41%|███████████████████████████████▉                                              | 9471/23086 [05:22<07:34, 29.97it/s]

In [ ]:
model_ft6 = train_model(model2, criterion, optimizer, exp_lr_scheduler2) # Epoch 8-12. We choose the final epoch save file ~ 12 epochs

Epoch 0/4
----------


100%|███████████████████████████████████████████████████████████████████████████| 11214/11214 [1:24:10<00:00,  2.22it/s]


train Loss: 0.5393 Acc: 0.7516


100%|█████████████████████████████████████████████████████████████████████████████| 23086/23086 [13:05<00:00, 29.38it/s]


val Loss: 0.6234 Acc: 0.6764

Epoch 1/4
----------


...

val Loss: 0.6208 Acc: 0.6766

Epoch 4/4
----------


100%|███████████████████████████████████████████████████████████████████████████| 11214/11214 [1:24:18<00:00,  2.22it/s]


train Loss: 0.4938 Acc: 0.7911


100%|█████████████████████████████████████████████████████████████████████████████| 23086/23086 [13:02<00:00, 29.51it/s]


val Loss: 0.6190 Acc: 0.6848

Training complete in 487m 9s
Best Val Acc: 0.6848


In [ ]:
torch.save({'model_state_dict': model_ft6.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                       }, 'Deit_Dense_AdamwFINAL_epoch12.pt')

## Testing


In [ ]:
# TEST of Deit_Dense_AdamwFINAL_epoch12.pt

In [ ]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model_ft6.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model_ft6(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 32:
        for i in range(32):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))

100%|█████████████████████████████████████████████████████████████████████████████| 24022/24022 [14:21<00:00, 27.89it/s]

Test Loss: 0.0000
Test Accuracy of non_sponsor: 75% (288188/380178)
Test Accuracy of sponsor: 66% (256670/388526)
Test Accuracy of 70% (544858/768704)
